In [1]:
#Check the enviroment that Jupyter is using:
#import sys
#sys.executable

#NOTE on scikit-learn: Is used for typical Machine Learning Algorihms and not Deep-learning.

import numpy
#Machine learning library:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#Image processing library:
from skimage import io, color
'''
Note on Tensorflow: Uses Nvidia drivers and CUDA for parallel execution: https://developer.nvidia.com/cuda-toolkit
Else Tensorflow will fall back using the CPU.
Also: We can use cuDNN for GPU acceleration: 
https://developer.nvidia.com/cudnn
'''
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
#Plotting library:
import matplotlib.pyplot as plt
#CSV handler: https://docs.python.org/3/library/csv.html 
import csv
#Web handler: https://pypi.org/project/requests/
import requests 

2023-09-21 14:13:13.091287: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-21 14:13:13.126486: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-21 14:13:13.127245: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 14:13:13.751660: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
dataset = [] 

#Open CSV file for reading and filter for evaluated artefacts: 
with open('DIME billeder.csv', mode='r', newline='') as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter=';')

    #Extract URL and Process fields:
    for row in csv_reader:
        process = row['process']
        thesaurus = row['thesaurus']
        #Filter for evaluated artefacts:
        if 'dime.find.process.evaluated' in process:
            #Only url of coins:
            if 'coin' in thesaurus:
                url = row['URL']
                data_point = {'url':url}
                dataset.append(data_point)

#Testing:
'''
for data_point in dataset:
    print('URL:', data_point['url'])
'''
'''
for data_point in dataset:
    print("URL:", data_point['url'])
    print("Process:", data_point['process'])
''' 

'\nfor data_point in dataset:\n    print("URL:", data_point[\'url\'])\n    print("Process:", data_point[\'process\'])\n'

In [4]:
'''
Display the top 10 first images in our dataset. 
Test purpose:
'''
for image in dataset:
    for x in range(10):
        print(image['url'])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

